# Scikit-learn
<img src="http://scikit-learn.org/stable/_images/scikit-learn-logo-notext.png" width=200 heigh=200></img>

Библиотека машинного обучения для Python с открытым исходным кодом. Позволяет решать задачи классификации, регрессии, кластеризации.

# Обучение моделей

В scikit-learn задачи обучения с учителем разделяются на кластеризацию и регрессию.
Каждая модель - отдельный класс с обязательными двумя методами:
* **`.fit(X, y)`** - обучение модели (настройка параметров, ...)
* **`.predict(X)`** - вычисление прогноза (у классификаторов так же может присутствовать метод **`.predict_proba(X)`**)

In [ ]:
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression

X, y = load_digits(return_X_y=True)

clf = LogisticRegression()
clf.fit(X, y)
clf.predict(X)

# Выбор моделей

Есть два основных подхода:
* отложенная выборка - выборка разделяется на две части (train/test) и обучение производится на части train, а оценка качества на test.
* кросс-валидация
<img src="https://habrastorage.org/files/b1d/706/e6c/b1d706e6c9df49c297b6152878a2d03f.png" width=400></img>

## Отложенная выборка

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, shuffle=True)

clf = LogisticRegression()
clf.fit(X_train, y_train)

accuracy_score(y_test, clf.predict(X_test))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier

classifiers = [LogisticRegression(), KNeighborsClassifier(), LinearSVC()]

for clf in classifiers:
    clf.fit(X_train, y_train)
    score = accuracy_score(y_test, clf.predict(X_test))
    clf_name = clf.__class__.__name__
    print('{: <32} :{:.4f}'.format(clf_name, score))

## Кросс-валидация

In [ ]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
import numpy as np

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

clf = LogisticRegression()
params={'C': np.logspace(-3, 3, 10)}

gs = GridSearchCV(clf, param_grid=params, cv=cv)
gs.fit(X_train, y_train)
gs.best_params_

In [ ]:
import pandas as pd
%matplotlib inline

pd.DataFrame(gs.cv_results_).set_index('param_C')['mean_test_score'].plot();

# Обработка данных

Помимо моделей обучения с учителем так же присутуствуют модели обучения буз учителя с интерфейсом:
* **`.fit(X)`** - настройка параметров
* **`.transform(X)`** - вычисление преобразования

In [ ]:
from sklearn.preprocessing import StandardScaler
scaller = StandardScaler()
scaller.fit(X_train)
scaller.transform(X_test)

Можно установить цепочку вычислений **`Pipeline`** из преобразований, а в конце модель.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

clf = Pipeline([
    ('scaller', StandardScaler()),
    ('PCA', PCA(n_components=12)),
    ('lr', LogisticRegression())
])

clf.fit(X_train, y_train)
accuracy_score(y_test, clf.predict(X_test))

## [sklearn_pandas](https://github.com/scikit-learn-contrib/sklearn-pandas)

Для `pandas.DataFrame` бывает полезно делать различные преобразования полей.
Например, мы хотим все поля перевести в числа. Это можно делать явно преобразованиями data frame. Так же можно воспользоваться 
оберткой `sklearn_pandas`.

In [ ]:
titanic_url = 'https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/train.csv'
titanic = pd.read_csv(titanic_url, index_col='PassengerId')
titanic.head(3)

In [ ]:
try:
    import sklearn_pandas
except ImportError:
    !pip install sklearn_pandas

from sklearn_pandas import DataFrameMapper, CategoricalImputer
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, Imputer

mapper = DataFrameMapper([
    ('Pclass', LabelBinarizer()),
    (['Age'], Imputer()),
    ('Embarked', [CategoricalImputer(), LabelBinarizer()]),
    ('Sex', LabelEncoder()),
    (['Fare', 'SibSp', 'Parch'], None)
])

mapper.fit_transform(titanic)[1]

In [ ]:
clf = Pipeline([
    ('data_frame_mapper', mapper),
    ('scaller', StandardScaler()),
    ('lr', LogisticRegression())
])

titanic_train, titanic_test = train_test_split(titanic, test_size=0.33, random_state=42, shuffle=True)

clf.fit(titanic_train, titanic_train['Survived'])
titanic_pred_test = clf.predict(titanic_test)

accuracy_score(titanic_test['Survived'], titanic_pred_test)

# [Catboost](https://tech.yandex.com/catboost/doc/dg/concepts/about-docpage)

Библиотека алгоритма градиентного бустинга от yandex. Отличается скоростью и качеством дефолтных параметров.

In [ ]:
try:
    from catboost import CatBoostClassifier
except ImportError:
    !pip install catboost
    from catboost import CatBoostClassifier

features = 'Pclass Sex Age SibSp Parch Fare'.split()
cat_features = [1]
clf = CatBoostClassifier(iterations=10, learning_rate=0.1, logging_level='Silent')
clf.fit(X=titanic_train[features], y=titanic_train['Survived'], cat_features=cat_features)

accuracy_score(titanic_test['Survived'], clf.predict(titanic_test[features]))

# [Keras](https://keras.io)
Библиотека для построения нейронных сетей с интерфейсом, близким к scikit-learn.

In [ ]:
try:
    import keras
except ImportError:
    !pip install --upgrade tensorflow
    !pip install keras
    import keras

In [ ]:
X, y = load_digits(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, shuffle=True)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

model = Sequential()
model.add(Dense(32, input_dim=64))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [ ]:
model.fit(X_train, y_train, epochs=50, batch_size=256, verbose=0)

pred_test = model.predict_classes(X_test)
accuracy_score(y_test, pred_test)

# Ссылки
* [документация scikit-learn](http://scikit-learn.org/stable/documentation.html)
* [пост с datacamp](https://www.datacamp.com/community/tutorials/machine-learning-python)
* [статья с хабра](https://habrahabr.ru/company/mlclass/blog/247751)
* [лекция из курса ODS в формате статьи на хабре](https://habrahabr.ru/company/ods/blog/322534)